In [1]:
import face_recognition
import cv2
import os
import json
import sys
import pickle

In [18]:
# import time
# def recognize_face(input_image_file):
#     start_time = time.time()

#     known_faces = []
#     known_names = []

#     # known_dir = "known_faces/"
#     # for filename in os.listdir(known_dir):
#     #     if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
#     #         image = face_recognition.load_image_file(os.path.join(known_dir, filename))
#     #         encodings = face_recognition.face_encodings(image)
#     #         if encodings:
#     #             known_faces.append(encodings[0])
#     #             known_names.append(filename.split('.')[0])
#     for filename in os.listdir("known_faces"):
#         image = face_recognition.load_image_file(f"known_faces/{filename}")
#         enc = face_recognition.face_encodings(image)
#         print("encodings",enc)
#         if enc:
#             known_faces.append(enc[0])
#             known_names.append(filename.split('.')[0])

#     with open("encodings.pkl", "wb") as f:
#          pickle.dump((known_faces, known_names), f)
#     input_image = face_recognition.load_image_file(input_image_file)
#     face_locations = face_recognition.face_locations(input_image, model="cnn")
#     input_encodings = face_recognition.face_encodings(input_image, face_locations)

#     if not input_encodings:
#         return {"status": "no_face_detected"}

#     for input_encoding in input_encodings:
#         results = face_recognition.compare_faces(known_faces, input_encoding)
#         if True in results:
#             matched_index = results.index(True)
#             end_time = time.time()
#             return {
#                 "status": "matched",
#                 "person": known_names[matched_index],
#                 "time_taken_sec": round(end_time - start_time, 3)
#             }

#     end_time = time.time()
#     return {
#         "status": "not_matched",
#         "time_taken_sec": round(end_time - start_time, 3)
#     }
# result = recognize_face('another_man.jpg')
# print(json.dumps(result))
# # # Command line usage 
# if __name__ == "__main__": 
#     if len(sys.argv) > 1: 
#         # image_path = sys.argv[1] 
#         result = recognize_face('another_man.jpg') 
#         print(json.dumps(result))
#     else: 
#         print('Usage: python face_recognition_script.py <image_path>')


import face_recognition
import cv2
import os
import json
import sys
import pickle
import time

ENCODING_FILE = "encodings.pkl"
KNOWN_DIR = "known_faces/dataset_images"

def load_known_faces():
    # Load cached encodings if exists
    if os.path.exists(ENCODING_FILE):
        with open(ENCODING_FILE, "rb") as f:
            return pickle.load(f)

    known_faces = []
    known_names = []

    for filename in os.listdir(KNOWN_DIR):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            path = os.path.join(KNOWN_DIR, filename)
            image = face_recognition.load_image_file(path)

            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(os.path.splitext(filename)[0])

    # Save cache
    with open(ENCODING_FILE, "wb") as f:
        pickle.dump((known_faces, known_names), f)

    return known_faces, known_names


def recognize_face(input_image_file):
    start_time = time.time()

    known_faces, known_names = load_known_faces()

    if not known_faces:
        return {"status": "no_known_faces"}

    input_image = face_recognition.load_image_file(input_image_file)

    # Faster model on CPU: "hog"
    face_locations = face_recognition.face_locations(input_image, model="hog")

    input_encodings = face_recognition.face_encodings(input_image, face_locations)

    if not input_encodings:
        return {"status": "no_face_detected"}

    for input_encoding in input_encodings:
        distances = face_recognition.face_distance(known_faces, input_encoding)

        best_match_index = distances.argmin()
        best_distance = distances[best_match_index]

        # Threshold (lower = stricter)
        if best_distance < 0.50:
            return {
                "status": "matched",
                "person": known_names[best_match_index],
                "confidence": round(1 - best_distance, 3),
                "time_taken_sec": round(time.time() - start_time, 3)
            }

    return {
        "status": "not_matched",
        "time_taken_sec": round(time.time() - start_time, 3)
    }


# CLI usage
if __name__ == "__main__":
    if len(sys.argv) > 1:
        image_path = sys.argv[1]
        result = recognize_face('new_img2.jpg')
        print(json.dumps(result))
    else:
        print("Usage: python script.py <image_path>")


{"status": "matched", "person": "14_2", "confidence": 1.0, "time_taken_sec": 0.504}
